In [117]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import xgboost

In [118]:
def get_data():
    train = pd.read_csv('/train.csv')
    test = pd.read_csv('/test.csv')
    y_train = train['satisfaction']
    train.drop('satisfaction', axis = 1, inplace = True)

    data = pd.concat([train, test]).reset_index(drop = True)
    data = pd.get_dummies(data)
    n_train = train.shape[0]
    
    # print('Missing values before: \n')
    # print(data.isna().sum().sum())
    
    put = data['Arrival Delay in Minutes'].mean()
    data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(put)

    imp_feats = ['Railway service',
            'Seat comfort',
            'Class_first-class',
            'Washroom service',
            'Mode of reservation_offline',
            'Ticket-collector service',
            'Distance Travelled',
            'Mode of reservation_online',
            'Age',
            'Baggage security score',
            'id',
            'Travel_frequent',
            'Class_second-class',
            'Cleanliness',
            'Compartment safety score',
            'Ease of Online booking',
            'Travel_non-frequent'
            ]
    
    data = data[imp_feats]

    # print('Missing values after: \n')
    # print(data.isna().sum().sum())
    
    X_train = data[:n_train]
    X_test = data[n_train:]
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
   
    return (X_train, y_train, X_test)
    
X_train, y_train, X_test = get_data()
print(X_train.shape, y_train.shape, X_test.shape)

(54931, 17) (54931,) (11834, 17)


In [119]:
def get_stacking():
    level0 = []
    level0.append(('cart', DecisionTreeClassifier()))
    level0.append(('rfc', RandomForestClassifier()))
    level0.append(('xgb', XGBClassifier()))

    level1 = RandomForestClassifier()
    
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv = 3)
    return model
get_stacking()

StackingClassifier(cv=3,
                   estimators=[('cart',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort='deprecated',
                      

In [120]:
def sub(name, score, preds):
    out = pd.DataFrame(
        {
            'satisfaction' : preds
        }
    )
    name = str(score) + '__' + name + '__new.csv'
    out.to_csv(name, index = False)

In [121]:
def get_models():
    models = {
        # 'lr' : LogisticRegression(),
        # 'knn' : KNeighborsClassifier(),
        'cart' : DecisionTreeClassifier(),
        'rfc' : RandomForestClassifier(),
        'stacking' : get_stacking(),
        'XGB': XGBClassifier(),
        # 'AdaBoost': AdaBoostClassifier(),
    }
    return models

models = get_models()
print(models)

{'cart': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'), 'rfc': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'stackin

In [122]:
X_train, y_train, X_test = get_data()
def evaluate(model):
    cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)

    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv = cv, n_jobs=-1, error_score='raise')
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    return (np.mean(scores), preds)

In [123]:
ans = []
for name, model in models.items():
  print(name, '...')
  score, preds = evaluate(model)
  ans.append((name, score, preds))
  print(score)

cart ...
0.9424854206240594
rfc ...
0.9639001626558178
stacking ...
0.9495306350533714
XGB ...
0.9544033806019772


In [124]:
# download all predictions
for a in ans:
  sub(a[0], a[1], a[2])

In [125]:
"""
cart ...
0.942382268635149
rfc ...
0.9638576906467617
stacking ...
0.9642399897644043
XGB ...
0.9541910026609082
AdaBoost ...
0.9384743357604365
"""

'\ncart ...\n0.942382268635149\nrfc ...\n0.9638576906467617\nstacking ...\n0.9642399897644043\nXGB ...\n0.9541910026609082\nAdaBoost ...\n0.9384743357604365\n'